# Задание: 
Для каждого кадра с камеры 1 нужно найти соответствующий (ближайший по времени) кадр с камеры 2. 
* Камеры записывают в формате (номер кадра, время его захвата) ~ (frame, timestamp).
* Камеры могут делать запись с разной частотой кадров в секунду и наличием шума (несоответствии времени кадра ожидаемому времени).

## Настройки/Гиперпараметры/Импорты

In [1]:
import time # для подсчёта времени работы
import numpy as np # для быстрой работы с массивами
from multiprocessing import Pool # для параллельного запуска (нужны процессы, а не потоки из multithreading/concurrent.futures)
from numba import njit # импорт компилятора для ускорения питона

In [2]:
cores_number = 2 # число ядер, на которые будет распараллеливаться выполнение кода
runs = 5 # число запусков для усреднения времени

## Вспомогательные функции

In [3]:
def make_timestamps(fps: int, st_ts: float, fn_ts: float) -> np.ndarray: # функция генерации записей с камеры
    """
    Создаёт np.array timestamps (массив времени записи кадра). 
    Этот массив дискретизирован по fps, но не равномерно.
    Возвращаемые timestamps отсортированы и уникальны.\n
    Parameters:
        * fps: среднее число кадров в секунду
        * st_ts: первый timestamp в последовательности (время первого кадра)
        * fn_ts: первый timestamp в последовательности (время последнего кадра) \n
    Returns:
        * np.ndarray: сгенерированные timestamps (массивы времени записи кадра)
    """
    timestamps = np.linspace(st_ts, fn_ts, int((fn_ts - st_ts) * fps)) # создаём временные записи из равномерного распределения
    timestamps += np.random.randn(len(timestamps)) # зашумляем кадры
    timestamps = np.unique(np.sort(timestamps)) # сортируем кадры по времени и оставляем только уникальные
    return timestamps

In [4]:
def calc_average_time(alg, data, runs: int=100) -> (float, np.ndarray): # функция для подсчёта среднего времени работы алгоритма
    """
    Функция для подсчёта среднего времени работы алгоритма.\n
    Parameters:
        * alg: алгоритм для тестирования
        * data: данные на вход алгоритма
        * runs: число запусков для усреднения времени
    Returns:
        * float: среднее время работы
        * np.ndarray: получившийся ответ
    """
    time_start = time.perf_counter() # замеряем время начала
    for i in range(runs): # делаем runs запусков
        answer = alg(*data) # вызываем алгоритм (*data для развёртывания переданных данных для алгоритма, если они в list-like формате)
        print(i, (time.perf_counter() - time_start)/(i+1)) # вывод среднего времени i-ой итерации (i+1 из-за нумерации с нуля)
    time_average = (time.perf_counter() - time_start) / runs # считаем среднее время
    return time_average, answer # возвращаем среднее время и получившийся ответ

In [5]:
# generate timestamps for the first camera
# timestamps1 = make_timestamps(30, time.time() - 100, time.time() + 300 * 2) # не 3600 (60 секунд * 60 минут), а лишь (10 * 60) = 10 минут
# generate timestamps for the second camera
# timestamps2 = make_timestamps(60, time.time() + 200, time.time() + 300 * 2.5) # не 3600 (60 секунд * 60 минут), а лишь (10 * 60) = 10 минут

timestamps1 = make_timestamps(30, time.time() - 100, time.time() + 900 * 2) # не 3600 (60 секунд * 60 минут), а лишь (10 * 60) = 10 минут
timestamps2 = make_timestamps(60, time.time() + 200, time.time() + 900 * 2.5) # не 3600 (60 секунд * 60 минут), а лишь (10 * 60) = 10 минут

In [6]:
timestamps1.shape

(57000,)

## Варианты решения

### 1) Глупо считаем разницу во времени между i-м кадром с первой камеры и всеми кадрами со второй камеры. В ответ записываем номер кадра со второй камеры с наименьшим модулем разности времени.

In [7]:
@njit #====================== использовать ли numba компилятор для функции ============================
def match_timestamps_naive(timestamps1: np.ndarray, timestamps2: np.ndarray) -> np.ndarray: 
# так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива 2n; сокращаем на константы и получаем n
# делаем для каждого элемента первого массива: - O(n)
#   из всех элементов второго массива вычитается элемент первого массива (считаем delta) за O(2n) ~ O(n)
#   в получившихся delta ищем наименьший элемент за O(2n) ~ O(n)
# итоговая сложность: O(размер_первого_массива * (размер_второго_массива + размер_второго_массива)) = O(n*(2n+2n)) = O(4n^2) ~ O(n^2)
    frames_count = timestamps1.shape[0] # число кадров на камере 1
    correspondence = np.zeros(frames_count, dtype=np.int32) # создаём массив под номера кадров с типом int32 для уменьшения потребляемой памяти

    #===================== вариант с циклом в np.arange ================================================
    # for frame in np.arange(frames_count): # идём по номерам кадров с первой камеры
    #--------------------- вариант с циклом в enumerate ------------------------------------------------
    for frame, frame_time in enumerate(timestamps1): # идём по кадрам
    #===================================================================================================
        #===================== вариант с сохранением delta =============================================
        # deltas = np.absolute(timestamps2 - timestamps1[frame]) # считаем разницу между временами
        # correspondence[frame] = deltas.argmin() # берём номер кадра с наименьшей разницей
        #--------------------- вариант без сохранения delta --------------------------------------------
        correspondence[frame] = np.absolute(timestamps2 - frame_time).argmin() # цикл с enumerate
        # correspondence[frame] = np.absolute(timestamps2 - timestamps1[frame]).argmin() # цикл с np.arange
        #===============================================================================================

    return correspondence

In [8]:
true_time, true_answer = calc_average_time(match_timestamps_naive, (timestamps1, timestamps2), runs=runs) # без njit, цикл в np.arange
# время работы либо 20 секунд, либо 70 и почему — не понятно 
# мб хеширование у ipynb файлов, так как уже замечалась странная работа с multiprocessing в notebook с функциями из того же notebook; в .py скрипте такое не наблюдается

0 16.7280714
1 16.088027750000002
2 16.68221266666667
3 16.507550925
4 16.37149692


<pre>
- naive no njit + np.arange ~ 20.0 - 70.0 секунд
- naive no njit + enumerate ~ 18.0 - 19.0 секунд
- naive njit    + np.arange ~ 15.5 - 15.8 секунд
- naive njit    + enumerate ~ 15.0 - 15.7 секунд
</pre>

### 2) Параллельная версия первого алгоритма

In [9]:
from func import match_timestamps_naive # импорт функции, так как в notebook multiprocessing навсегда зависает при вызове функции из этого же notebook (???)

In [10]:
def match_timestamps_naive_parallel(timestamps1: np.ndarray, timestamps2: np.ndarray, func, cores_number: int) -> np.ndarray: 
# так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива 2n; сокращаем на константы и получаем n
# делаем для каждого элемента первого массива с распараллеливанием: - O(n/cores_number)
#   из всех элементов второго массива вычитается элемент первого массива (считаем delta) за O(2n) ~ O(n)
#   в получившихся delta ищем наименьший элемент за O(2n) ~ O(n)
# итоговая сложность: O(размер_первого_массива / число_ядер * (размер_второго_массива + размер_второго_массива)) = O(n/cores_number*(2n+2n)) = O(4/cores_number*n^2) ~ O(n^2/cores_number)
    frames_count = timestamps1.shape[0] # число кадров с первой камеры
    with Pool(cores_number) as p: # запускаем cores_number процессов
        correspondence = np.concatenate(p.starmap(match_timestamps_naive, [(timestamps1[int(i*frames_count/cores_number):int((i+1)*frames_count/cores_number)], timestamps2) for i in np.arange(cores_number)]), dtype=np.int32)
        #  - каждому процессу отправляем равную часть кадров с первой камеры и все кадры со второй
        #  - конкатенируем вернувшиеся из процессов массивы с индексами
        # не обязательно передавать chunksize, так как распараллеливание уже идёт по кейсам/инпутам для функций
        # dtype=np.int32 для уменьшения потребляемой памяти и ускорения работы

    return correspondence

In [11]:
time_naive_parallel, answer_naive_parallel = calc_average_time(match_timestamps_naive_parallel, (timestamps1, timestamps2, match_timestamps_naive, cores_number), runs=runs) # подсчёт среднего времени (с njit и enumerate циклом)
# время примерно 40 секунд (если считать, что непараллельная версия выполняется за 70, то всё ок)

0 10.6483499
1 11.165950449999997
2 11.167743366666665
3 11.054101725000002
4 10.969213000000002


In [12]:
np.all(answer_naive_parallel==true_answer) # проверка, что параллельная функция отработала корректно

True

<pre>
- parallel naive no njit + np.arange ~ 40.0 - 41.0 секунд
- parallel naive no njit + enumerate ~ 41.0 - 43.0 секунд
- parallel naive njit    + np.arange ~ 11.2 - 12.0 секунд
- parallel naive njit    + enumerate ~ 10.0 - 11.2 секунд
</pre>

### 3) Идём по последовательности кадров с первой камеры, запоминанием найденное соответствие со второй (номер delta, после которого разница во времени начинают увеличиваться). На следующем шаге по кадрам на второй камере начинаем идти с найденного соответствия на предыдущем шаге (так как кадры отсортированы в порядке неубывания - нет смысла рассматривать кадры, что заведомо имеют большую delta из-за увеличения времени на первой камере).

In [18]:
@njit #====================== использовать ли numba компилятор для функции ============================
def match_timestamps_iterated(timestamps1: np.ndarray, timestamps2: np.ndarray, cam2_frame: int=0) -> np.ndarray: 
# так как в задании первая камера пишет в ~30 fps -> размер первого массива n; вторая камера пишет в ~60 fps -> размер второго массива 2n; сокращаем на константы и получаем n

# ищем начальный кадр для второй камеры - O(2n), однако реальная сложность меньше, так как cam2_frame уже найден эвристически с учётом числа кадров на первой и второй камере
# делаем для каждого элемента (времени) с первой камеры: - O(n)
#   вычитаем из соответствующего времени кадра второй камеры значение времени кадра с первой камеры - O(2n) в худшем случае, O(2) ~ O(1) в среднем, так как кадры распределены равномерно 
#       если дельта уменьшилась - переходим к следующему кадру на второй камере и так далее, пока дельта уменьшается
#       если дельта не уменьшилась - возвращаем предыдущий кадр, как самый близкий по времени
# итоговая сложность: O(размер_второго_массива + размер_первого_массива * размер_второго_массива) ~ [в среднем] ~ O(размер_второго_массива + размер_первого_массива * 1) ~ O(n)

    frames_count = timestamps1.shape[0] # число кадров на камере 1
    max_frame = timestamps2.shape[0] - 1 # максимальный номер кадра, что может соответствовать кадру с первой камеры (-1 из-за индексации с нуля)
    correspondence = np.zeros(frames_count, dtype=np.int32) # создаём массив под номера кадров с типом int32 для уменьшения потребляемой памяти

    if 0 > cam2_frame or cam2_frame > max_frame: # защита от дурака с cam2_frame
        cam2_frame = 0 # зануление начального кадра второй камеры

    # подбираем кадр на второй камере, с которого будем начинать (эвристически уже передали ожидаемый cam2_frame, нужно его лишь правильно сместить, если он "обогнал" кадры с камеры 1)
    delta = np.abs(timestamps2[cam2_frame] - timestamps1[0]) # текущая разность времени
    while np.abs(timestamps2[max(0, cam2_frame-1)] - timestamps1[0]) < delta: # если дельта уменьшается при предыдущем кадре, то переходим на него (условие — строго меньше, так как времена всех кадров уникальны)
        cam2_frame -= 1 # переходим на предыдущий кадр (без max, так как при нуле мы бы сюда не зашли в цикл)
        delta = np.abs(timestamps2[cam2_frame] - timestamps1[0]) # обновляем текущее значение delta (без max, так как при нуле мы бы не зашли в цикл)
    # после этого цикла мы либо находимся на оптимальном соответствии первого кадра первой камеры с кадром со второй камеры, либо оптимум — далее по времени (но не раньше!)

    for frame in np.arange(frames_count): # идём по номерам кадров с первой камеры
        delta = np.abs(timestamps2[cam2_frame] - timestamps1[frame]) # текущая разность времени
        while np.abs(timestamps2[min(cam2_frame+1, max_frame)] - timestamps1[frame]) < delta: # если при переходе на следующий кадр дельта (разница времени) уменьшилась, то обновляем delta и соответствующий кадр на второй камере 
            cam2_frame += 1 # переходим на следующий кадр
            delta = np.abs(timestamps2[cam2_frame] - timestamps1[frame])  # обновляем delta
        correspondence[frame] = cam2_frame # если дельта перестала уменьшаеться — записываем найденный кадр

    return correspondence

In [19]:
answer_iterated = match_timestamps_iterated(timestamps1, timestamps2)

### 4) Параллельная реализация третьего алгоритма